In [1]:
import numpy as np
import pandas as pd
from parse import parse
import re
from aocd import get_data
from numba import njit
import sys
sys.setrecursionlimit(10000)
data = get_data(year=2018, day=20)

In [2]:
tests = [*{
    '^WNE$': 3,
    '^ENWWW(NEEE|SSE(EE|N))$': 10,
    '^ENNWSWW(NEWS|)SSSEEN(WNSE|)EE(SWEN|)NNN$': 18,
    '^ESSWWN(E|NNENN(EESS(WNSE|)SSS|WWWSSSSE(SW|NNNE)))$': 23,
    '^WSSEESWWWNW(S|NENNEEEENN(ESSSSW(NWSW|SSEN)|WSWWN(E|WWS(E|SS))))$': 31,
}.items()]

In [11]:
class Room(object):
    def __init__(self, N=None, S=None, E=None, W=None):
        self.N = N
        self.S = S
        self.E = E
        self.W = W
        
    def __hash__(self):
        return id(self)
    
    @property
    def edges(self):
        return {*filter(None, (self.N, self.S, self.E, self.W))}
        
    def depth(self, d=0, seen=set()):
        return max([e.depth(d + 1, seen | self.edges) for e in self.edges - seen], default=d)
    
    def part2(self, d=1, limit=0, seen=set()):
        count = len(self.edges - seen) * (d >= limit)
        return sum([e.part2(d + 1, limit, seen | self.edges) for e in self.edges - seen]) + count
        

In [4]:
ref = {'N': 'S', 'S': 'N', 'E': 'W', 'W': 'E'}

In [5]:
def graph(base, path):
    X = base
    for c in path:
        if c in ref:
            if X.__getattribute__(c) is None:
                X.__setattr__(c, Room(**{ref[c]: X}))
            X = X.__getattribute__(c)
        elif c == '|':
            X = base
        elif c == '(':
            graph(X, path)
        elif c == ')':
            break

In [6]:
def part1(data):
    X = Room()
    graph(X, iter(data))
    return X.depth()

In [7]:
for d, v in tests:
    result = part1(d)
    passfail = ['Fail', 'Pass'][int(result == v)]
    print(f"{passfail} {v:4d}: {d}")

Pass    3: ^WNE$
Pass   10: ^ENWWW(NEEE|SSE(EE|N))$
Pass   18: ^ENNWSWW(NEWS|)SSSEEN(WNSE|)EE(SWEN|)NNN$
Pass   23: ^ESSWWN(E|NNENN(EESS(WNSE|)SSS|WWWSSSSE(SW|NNNE)))$
Pass   31: ^WSSEESWWWNW(S|NENNEEEENN(ESSSSW(NWSW|SSEN)|WSWWN(E|WWS(E|SS))))$


In [8]:
part1(data)

4186

In [12]:
def part2(data, limit=1000):
    X = Room()
    graph(X, iter(data))
    return X.part2(limit=limit)

In [14]:
part2(data)

8466